## Conversational AI

In [2]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [3]:
load_dotenv(override=True)

api_key = os.getenv("OPENAI_API_KEY")
openrouter_api_key = os.getenv("OPENROUTER_API_KEY")
groq_api_key = os.getenv("GROQ_API_KEY")

# Check the API key
if (not api_key) and (not openrouter_api_key) and (not groq_api_key):
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif (not api_key.startswith("sk-proj-")) and (not openrouter_api_key.startswith("sk-")) and (not groq_api_key.startswith("gsk-")):
    print("An API key was found, but it is in the wrong format; please check you're using the right key - see troubleshooting notebook")
elif (api_key.strip() != api_key) and (openrouter_api_key.strip() != openrouter_api_key) and (groq_api_key.strip() != groq_api_key):
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [6]:
OLLAMA_BASE_URL = "http://localhost:11434/v1"
ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')
GPT_MODEL = "gpt-5-nano"
LLAMA_MODEL = "llama3.2"

openai = OpenAI()

In [5]:
system_message = "You are a helpful assistant"

In [19]:

def chat(message, history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = ollama.chat.completions.create(model=LLAMA_MODEL, messages=messages)
    return response.choices[0].message.content

In [16]:
gr.ChatInterface(fn=chat).launch()

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


In [24]:
## Using OpenAI model with streaming


def chat_with_streaming(message, history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    stream = openai.chat.completions.create(model=GPT_MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ""
        yield response

In [25]:
gr.ChatInterface(fn=chat_with_streaming).launch()

* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.


In [26]:
system_message = "You are a helpful assistant in a clothes store. You should try to gently encourage \
the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off. \
For example, if the customer says 'I'm looking to buy a hat', \
you could reply something like, 'Wonderful - we have lots of hats - including several that are part of our sales event.'\
Encourage the customer to buy hats if they are unsure what to get."

In [27]:
gr.ChatInterface(fn=chat_with_streaming).launch()

* Running on local URL:  http://127.0.0.1:7867
* To create a public link, set `share=True` in `launch()`.


In [28]:
def chat_with_streaming(message, history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    relevant_system_message = system_message
    if 'belt' in message.lower():
        relevant_system_message += " The store does not sell belts; if you are asked for belts, be sure to point out other items on sale."
    
    messages = [{"role": "system", "content": relevant_system_message}] + history + [{"role": "user", "content": message}]

    stream = openai.chat.completions.create(model=GPT_MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [29]:
gr.ChatInterface(fn=chat_with_streaming).launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7868
* To create a public link, set `share=True` in `launch()`.
